In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import random
import pickle
import argparse
from copy import deepcopy
from IPython.display import clear_output

from l_system import *
from utils import *

In [2]:
l_system=L_system()
l_system.update_vocabulary({'X','F','-','+','[',']'})
#l_system.add_rule('X', 'F+[[X]-X]-F[-FX]+X',verbose=True)
l_system.add_rule('X', 'F[+X]-F',verbose=True)
l_system.add_rule('F', 'FF',verbose=True)
tree_interpreter=Tree_interpreter(plus_angle=np.pi/8,minus_angle=-np.pi/6,randomize_angle=False)
n_iter=6

In [3]:
IMG_WIDTH = 130
IMG_HEIGHT = IMG_WIDTH
n_trees=40
def get_offspring(ls,ti,n_iter):
    images=[]
    ls_offspring=[ls]
    ti_offspring=[ti]
    n_iter_offspring=[n_iter]
    #keep also the 
    w=ls.evolution('X',n_iter)
    segments=ti.render(w)
    im=draw_tree(segments,im_size=IMG_WIDTH)
    im.save('images/tree.png')
    file = open("images/tree.png", "rb")
    png_image = file.read()
    images.append(png_image)
    print('generate offspring...')
    for i in range(n_trees-1):
        ls_copy=deepcopy(ls)
        ti_copy=deepcopy(ti)
        ls_copy.point_mutation('X',0.04,immune={'[',']'})
        ls_copy.perm_mutation('X',0.05)
        ls_copy.loss_mutation('X',0.05,immune={'[',']'})
        ls_copy.add_mutation('X',0.02,words={'[X]','[F]'},immune={'[',']'})
        ti_copy.angle_mutation()
        ls_offspring.append(ls_copy)
        ti_offspring.append(ti_copy)
        r=np.random.rand()
        if r<0.1:
            n=min(8,n_iter+1)
        elif r>0.9:
            n=max(2,n_iter-1)
        else:
            n=n_iter
        n_iter_offspring.append(n)
        w=ls_copy.evolution('X',n)
        im=draw_tree(ti_copy.render(w),im_size=IMG_WIDTH)
        im.save('images/tree.png')
        file = open("images/tree.png", "rb")
        png_image = file.read()
        images.append(png_image)
    return images,ls_offspring,ti_offspring,n_iter_offspring


In [4]:
COLS = 5
ROWS = 8

def on_click(b):
    global ls_offspring,ti_offspring,n_iter_offspring
    clear_output()
    number_string=b.description.split(' ')[0]
    tree_number=int(number_string)
    print('selected tree: '+str(tree_number)+', with gene code: '+str(ls_offspring[tree_number].rule['X'])+' and iterations: '+str(n_iter_offspring[tree_number]))
    IMAGES,ls_offspring,ti_offspring,n_iter_offspring=get_offspring(ls_offspring[tree_number],ti_offspring[tree_number],n_iter_offspring[tree_number])
    show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=IMG_WIDTH,IMG_HEIGHT=IMG_HEIGHT)
    
import ipywidgets as widgets
import functools


def show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=200,IMG_HEIGHT=200):
    #clear_output()
    rows = []
    for row in range(ROWS):
        cols = []
        for col in range(COLS):
            index = row * COLS + col
            image = widgets.Image(
                value=IMAGES[index], width=IMG_WIDTH, height=IMG_HEIGHT
            )
            if index==0:
                add_string=' ancestor'
            else:
                add_string=' '
            button = widgets.Button(description=str(index)+add_string)
            button.on_click(on_click)
            box = widgets.VBox([image, button])
            cols.append(box)
        # Create a horizontal layout box, grouping all the columns together
        rows.append(widgets.HBox(cols))
    # Create a vertical layout box, grouping all the rows together
    result = widgets.VBox(rows)
    display(result)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

In [5]:
IMAGES,ls_offspring,ti_offspring,n_iter_offspring=get_offspring(l_system,tree_interpreter,n_iter)
show_offsprings(IMAGES,ROWS,COLS,IMG_WIDTH=IMG_WIDTH,IMG_HEIGHT=IMG_HEIGHT)

selected tree: 32, with gene code: X[FX]-XX and iterations: 8
generate offspring...


A Jupyter Widget